In [21]:
# import libraries
import numpy as np
import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# Import and process the data

In [22]:
# import dataset (comes with colab!)
data = pd.read_csv('mnist_train_small .csv',delimiter=',')

data_np = data.to_numpy()
labels = data_np[:, 0]
data = data_np[:, 1:]

In [23]:
# Normalize the data 

In [24]:
# normalize the data to a range of [0 1]
dataNorm = data / np.max(data)

In [25]:
# Create train/test groups using DataLoader

In [26]:
# Step 1: convert to tensor
dataT   = torch.tensor( dataNorm ).float()
labelsT = torch.tensor( labels ).long() # long = int64

# Step 2: use scikitlearn to split the data
train_data,test_data, train_labels,test_labels = train_test_split(dataT, labelsT, test_size=.1)


# Step 3: convert into PyTorch Datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Step 4: translate into dataloader objects
batchsize    = 32
train_loader = DataLoader(train_data,batch_size=batchsize,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

In [27]:
# Create the DL model

In [28]:
def createTheMNISTNet(nUnits, nLayers):
    class mnistNet(nn.Module):
        def __init__(self):
            super().__init__()
            # create dictionary to store the layers
            self.layers = nn.ModuleDict()
            self.nLayers = nUnits  
            
            ### input layer
            self.layers['input'] = nn.Linear(784, nUnits)
            
            ### hidden layers
            for i in range(nLayers):
                self.layers[f'hidden{i}'] = nn.Linear(nUnits, nUnits)
            ### output layer
            self.layers['output'] = nn.Linear(nUnits, 10)
            
        # forward pass reste identique
        def forward(self, x):
            # input layer
            x = F.relu(self.layers['input'](x))
            # hidden layers
            for i in range(self.nLayers):
                x = F.relu(self.layers[f'hidden{i}'](x))
            # return output layer
            x = self.layers['output'](x)
            return torch.log_softmax(x, axis=1)
    
    # create the model instance
    net = mnistNet()
    
    # loss function
    lossfun = nn.NLLLoss()
    # optimizer
    optimizer = torch.optim.SGD(net.parameters(), lr=.01)
    return net, lossfun, optimizer

In [29]:
# generate an instance of the model and inspect it
nUnitsPerLayer = 12
nLayers = 2
net = createTheMNISTNet(nUnitsPerLayer,nLayers)
net

(mnistNet(
   (layers): ModuleDict(
     (input): Linear(in_features=784, out_features=12, bias=True)
     (hidden0): Linear(in_features=12, out_features=12, bias=True)
     (hidden1): Linear(in_features=12, out_features=12, bias=True)
     (output): Linear(in_features=12, out_features=10, bias=True)
   )
 ),
 NLLLoss(),
 SGD (
 Parameter Group 0
     dampening: 0
     differentiable: False
     foreach: None
     fused: None
     lr: 0.01
     maximize: False
     momentum: 0
     nesterov: False
     weight_decay: 0
 ))